In [16]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

print('hello')
# 버전 확인
print(f"pandas 버전: {pd.__version__}")
print(f"numpy 버전: {np.__version__}")


hello
pandas 버전: 2.3.3
numpy 버전: 2.4.0


In [17]:

# 리스트로 Series 생성
sales = pd.Series([120, 150, 180, 200, 160])
print("매출 데이터:")
print(sales)
print(f"\n타입: {type(sales)}")

# NumPy 배열
numpy_array = np.array([10, 20, 30, 40, 50])
print("\nNumPy 배열:", numpy_array)
print("타입:", type(numpy_array))
print("데이터 타입:", numpy_array.dtype)

# Plotly
fig = go.Figure()
fig.show()

매출 데이터:
0    120
1    150
2    180
3    200
4    160
dtype: int64

타입: <class 'pandas.core.series.Series'>

NumPy 배열: [10 20 30 40 50]
타입: <class 'numpy.ndarray'>
데이터 타입: int64


In [18]:
#현재 작업 경로 확인
import os
print(os.getcwd())

#파일 불러오기
df = pd.read_csv('rd/union1.csv', encoding='utf-8')
print("cp949로 로드 성공!")

c:\Users\Administrator\Documents\workspace\budongsan3_project\ysy


FileNotFoundError: [Errno 2] No such file or directory: 'rd/union1.csv'

In [ ]:
#데이터 살펴보기

df.head()
df.info()
df.describe()
df.shape
df.columns.tolist()

⭐목표 정의

계약일의 흐름에 따라 부동산 실거래가가 어떻게 변하는지 자치구명, 법정동명 차원에서 분석

In [ ]:
#계약일을 datetime형식으로 변환
df['계약일'] = pd.to_datetime(df['계약일'], format= '%Y%m%d')
df['물건금액'] = round(df['물건금액'], 0)

print(df['계약일'].head())
print(df['물건금액'].head())

In [ ]:

#자치구-법정동 기준
trend_detail = df.groupby(['계약일','자치구명','법정동명'])['물건금액'].mean().round(0).unstack(level=['자치구명','법정동명']).sort_index(axis=1)

#자치구별 기준
trend_gu = trend_detail.groupby(level='자치구명', axis=1).mean().round(0)

#전체 강남3구 기준
trend_total = df.groupby(['계약일'])['물건금액'].mean().round(0)  
overall_avg = trend_total.mean().mean()

print(f"전체 강남3구 기준 평균거래금액: {overall_avg:,.2f}만 원")

print(trend_detail.head())
print(trend_total.head())

In [ ]:
# 인덱스를 datetime으로 변경⭐⭐⭐ 
# groupby하면서 '계약일'이 인덱스가 되면서 datetime이 풀렸을 가능성이 큼
trend_detail.index = pd.to_datetime(trend_detail.index)
trend_gu.index = pd.to_datetime(trend_gu.index)
trend_total.index = pd.to_datetime(trend_total.index)

#구-동 일별 데이터 ==> 월별로 resampling하기
trend_detail_monthly = trend_detail.resample('MS').mean()

# 구별 데이터를 일별 ==> 월별로 resampling하기
trend_gu_monthly = trend_gu.resample('MS').mean()

# 강남3구 데이터를 일별 ==> 월별로 resampling하기
trend_total_monthly = trend_total.resample('MS').mean()

print(trend_detail_monthly.head())
print(trend_gu_monthly.head())
print(trend_total_monthly.head())

print(f"시구별 데이터 행 개수: {len(trend_detail_monthly)}")
print(f"구별 데이터 행 개수: {len(trend_gu_monthly)}")
print(f"강남3구 전체 평균 행 개수: {len(trend_total_monthly)}")


In [ ]:
# # 데이터가 없는 달(NaN) 제거
# trend_detail_monthly = trend_detail_monthly.dropna()
# trend_gu_monthly = trend_gu_monthly.dropna()
# trend_total_monthly = trend_total_monthly.dropna()

# print(trend_detail_monthly.head())

In [ ]:
print(f"전체 행 개수: {len(trend_total)}")

In [ ]:
###시행착오 1:  x축 간격 조정 필요, 각 꺾은선 색상 표시 

# import plotly.graph_objects as go

# # 1. 인덱스 변환 (Period -> Timestamp)
# # Plotly는 '기간'보다 '시점' 데이터를 정확히 인식합니다.
# plot_gu = trend_gu_monthly.copy()
# plot_total = trend_total_monthly.copy()

# if hasattr(plot_gu.index, 'to_timestamp'):
#     plot_gu.index = plot_gu.index.to_timestamp()
# if hasattr(plot_total.index, 'to_timestamp'):
#     plot_total.index = plot_total.index.to_timestamp()

# # 2. 도표 객체 생성
# fig = go.Figure()

# # 3. 구별 월별 추이 선 추가 (연한 색상으로 배경 처리)
# for gu in plot_gu.columns:
#     fig.add_trace(go.Scatter(
#         x=plot_gu.index,
#         y=plot_gu[gu],
#         mode='lines',
#         name=gu,
#         line=dict(width=1),
#         opacity=0.2, # 구별 선은 아주 연하게 해서 겹쳐도 복잡하지 않게 함
#         hovertemplate='%{x|%Y-%m}<br>' + gu + ': %{y:,.0f}만 원<extra></extra>'
#     ))

# # 4. 서울 전체 평균 선 추가 (진한 색상으로 강조)
# fig.add_trace(go.Scatter(
#     x=plot_total.index,
#     y=plot_total.values,
#     mode='lines+markers', # 점을 추가해서 월별 데이터임을 명시
#     name='<b>⭐ 서울 전체 평균</b>',
#     line=dict(color='firebrick', width=4),
#     marker=dict(size=8, symbol='circle'),
#     hovertemplate='<b>%{x|%Y-%m} 평균: %{y:,.0f}만 원</b><extra></extra>'
# ))

# # 5. 레이아웃 및 디자인 설정
# fig.update_layout(
#     title=dict(
#         text='<b>서울시 자치구별 월별 평균 거래가 추이</b>',
#         x=0.5, font=dict(size=22)
#     ),
#     xaxis_title='계약월',
#     yaxis_title='평균 금액 (만 원)',
#     template='plotly_white', # 배경을 하얗고 깔끔하게
#     hovermode='x unified',     # 마우스를 대면 그 달의 모든 구 데이터를 한눈에 보여줌
    
#     xaxis=dict(
#         dtick="M3",            # 3개월 단위로 눈금 표시
#         tickformat="%y년 %m월"  # 축 표시 형식 (23년 01월)
#     ),
#     yaxis=dict(tickformat=',d'), # y축 숫자 콤마 표시
#     legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5)
# )

# fig.show()

In [ ]:
print(f"전체 행 개수: {len(plot_total)}")

In [ ]:
# ###시행착오3: 그래프를 합쳐서 봐야함

# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# # [핵심] 반드시 '_monthly'가 붙은 변수를 사용해야 합니다!
# # 만약 이 변수가 없다면 위에서 만든 trend_gu_monthly를 여기 대입하세요.
# plot_gu = trend_gu_monthly.copy() 

# # Plotly 인식용 타임스탬프 변환
# if hasattr(plot_gu.index, 'to_timestamp'):
#     plot_gu.index = plot_gu.index.to_timestamp()

# # 4행 1열 서브플롯 생성
# fig = make_subplots(
#     rows=4, cols=1, 
#     subplot_titles=("강남구 월별 가격 추이", "송파구 월별 가격 추이", "서초구 월별 가격 추이", "⭐ 강남 3구 전체 평균"),
#     shared_xaxes=False,
#     vertical_spacing=0.1
# )

# targets = ['강남구', '송파구', '서초구']
# colors = ['royalblue', 'forestgreen', 'darkorange']

# # 꺾은선 그래프 추가
# for i, gu in enumerate(targets):
#     fig.add_trace(
#         go.Scatter(
#             x=plot_gu.index, 
#             y=plot_gu[gu], 
#             mode='lines+markers', # 이제 한 달에 점이 딱 하나씩만 찍힐 겁니다
#             line=dict(color=colors[i], width=2),
#             marker=dict(size=6)
#         ),
#         row=i+1, col=1
#     )

# # 강남 3구 전체 평균 (이미 월별로 계산된 값을 사용)
# if '강남3구_전체' not in plot_gu.columns:
#     plot_gu['강남3구_전체'] = plot_gu[targets].mean(axis=1)

# fig.add_trace(
#     go.Scatter(
#         x=plot_gu.index, 
#         y=plot_gu['강남3구_전체'], 
#         mode='lines+markers',
#         line=dict(color='firebrick', width=3),
#         marker=dict(size=7, symbol='diamond')
#     ),
#     row=4, col=1
# )

# # 디자인 및 X축 설정
# fig.update_layout(height=1200, template='plotly_white', showlegend=False)
# fig.update_xaxes(tickformat="%y년 %m월", dtick="M3", tickangle=45)
# fig.update_yaxes(tickformat=',d')

# print(f"현재 그래프에 사용된 데이터 행 개수: {len(plot_gu)}") # 이 숫자가 30~40 사이여야 합니다!
# fig.show()

In [ ]:
# import plotly.graph_objects as go

# # 1. 데이터 복사 및 타임스탬프 변환
# plot_gu = trend_gu_monthly.copy()
# if hasattr(plot_gu.index, 'to_timestamp'):
#     plot_gu.index = plot_gu.index.to_timestamp()

# # 2. 강남 3구 통합 데이터 계산 (이미 되어있다면 생략 가능)
# targets = ['강남구', '송파구', '서초구']
# if '강남3구_전체' not in plot_gu.columns:
#     plot_gu['강남3구_전체'] = plot_gu[targets].mean(axis=1)

# # 3. 단일 도표 객체 생성
# fig = go.Figure()

# # 4. 각 구별 꺾은선 추가 (강남, 송파, 서초)
# colors = {'강남구': 'royalblue', '송파구': 'forestgreen', '서초구': 'darkorange'}

# for gu in targets:
#     fig.add_trace(go.Scatter(
#         x=plot_gu.index,
#         y=plot_gu[gu],
#         mode='lines',
#         name=gu,
#         line=dict(color=colors[gu], width=2),
#         hovertemplate='%{x|%Y-%m}<br>' + gu + ': %{y:,.0f}만 원<extra></extra>'
#     ))

# # 5. 강남 3구 전체 평균 선 추가 (가장 굵게 강조)
# fig.add_trace(go.Scatter(
#     x=plot_gu.index,
#     y=plot_gu['강남3구_전체'],
#     mode='lines',
#     name='<b>⭐ 강남 3구 전체 평균</b>',
#     line=dict(color='firebrick', width=4), # 주인공은 가장 굵게!
#     hovertemplate='<b>%{x|%Y-%m} 통합 평균: %{y:,.0f}만 원</b><extra></extra>'
# ))

# # 6. 전체 디자인 및 X축 날짜 설정
# fig.update_layout(
#     title=dict(
#         text='<b>강남 3구(강남·송파·서초) 및 통합 월별 가격 추이 비교</b>',
#         x=0.5, font=dict(size=22)
#     ),
#     # 1. 그래프 전체의 가로(width)와 세로(height) 크기 설정
#     width=1200,   # 기존보다 가로 길이를 충분히 늘림 (기본값은 보통 700~1000)
#     height=700,   # 세로 길이는 기존 유지
    
#     xaxis=dict(title=dict(text='계약월', standoff=30)),
#     yaxis_title='평균 거래가 (만 원)',
#     template='plotly_white',
#     hovermode='x unified',
    
#     # 범례를 하단 가로 방향으로 설정
#     legend=dict(
#         orientation="h",   # 가로 방향 (Horizontal)
#         yanchor="top",   # 범례 상단을 기준으로 위치 지정
#         y=-0.15,   # Y축 아래로 충분히 내림 (값이 작을수록 아래로 내려감)
#         xanchor="center",   # 범례 중앙을 기준으로
#         x=0.5   # 그래프 중앙에 배치
#     ),
# )

# # 1. 데이터의 시작과 끝 날짜 파악
# start_date = plot_gu.index.min()
# end_date = plot_gu.index.max()

# # 2. X축 설정 업데이트
# fig.update_xaxes(
#     range=[start_date, end_date], # 데이터가 있는 시작일과 종료일로 범위 고정
#     tickvals=plot_gu.index,
#     ticktext=[f"{m.month}월" for m in plot_gu.index], # '01월' 대신 '1월'로 변환
#     tickformat="%m월", 
#     dtick="M1", 
#     tickangle=0,
# )

# # 2. 연도 구분선 추가 (24년, 25년 시작 지점)

# # 흐린 회색 점선으로 설정하여 배경처럼 보이게 합니다.
# years = [2024, 2025, 2026]
# for year in years:
#     # 해당 연도 1월 1일 위치에 선 추가
#     year_start = f"{year}-01-01"

#     # 데이터 범위 내에 있는 연도만 선을 긋습니다.
#     if plot_gu.index.min() <= pd.to_datetime(year_start) <= plot_gu.index.max():
#         fig.add_vline(
#             x=year_start,
#             line_width=1.5,
#             line_dash="solid",              # 'dash'에서 'solid'(실선)로 변경
#             line_color="rgba(200, 200, 200, 0.5)" # 더 흐린 회색 (숫자가 높을수록 밝음)
#         )

#         # 선 위에 '24년', '25년' 텍스트 추가 (선택 사항)
#         fig.add_annotation(
#             x=year_start, y=1, yref="paper",
#             text=f"<b>{year}년</b>",
#             showarrow=False, font=dict(color="gray", size=12),
#             xanchor="center", yanchor="bottom"
#         )

# # Y축 숫자 콤마 설정
# fig.update_yaxes(tickformat=',d')

# fig.show()

In [ ]:
### 메인 그래프 함수로 재사용 가능하게 변수에 저장 ###

import plotly.graph_objects as go # 시각화 도구를 가져옵니다.
import pandas as pd # 데이터 처리를 위해 판다스를 가져옵니다.

def draw_trend_graph(data): # [추가] 전체 코드를 'draw_trend_graph'라는 이름의 변수(함수)로 지정합니다.
    # 1. 데이터 복사 및 타임스탬프 변환
    plot_gu = data.copy() # 입력받은 데이터를 복사하여 사용합니다.
    if hasattr(plot_gu.index, 'to_timestamp'):
        plot_gu.index = plot_gu.index.to_timestamp()

    # 2. 강남 3구 통합 데이터 계산
    targets = ['강남구', '송파구', '서초구']
    if '강남3구_전체' not in plot_gu.columns:
        plot_gu['강남3구_전체'] = plot_gu[targets].mean(axis=1)

    # 3. 단일 도표 객체 생성
    fig = go.Figure()

    # 4. 각 구별 꺾은선 추가
    colors = {'강남구': 'royalblue', '송파구': 'forestgreen', '서초구': 'darkorange'}
    for gu in targets:
        fig.add_trace(go.Scatter(
            x=plot_gu.index,
            y=plot_gu[gu],
            mode='lines',
            name=gu,
            line=dict(color=colors[gu], width=2),
            hovertemplate='%{x|%Y-%m}<br>' + gu + ': %{y:,.0f}만 원<extra></extra>'
        ))

    # 5. 강남 3구 전체 평균 선 추가 (강조)
    fig.add_trace(go.Scatter(
        x=plot_gu.index,
        y=plot_gu['강남3구_전체'],
        mode='lines',
        name='<b>⭐ 강남 3구 전체 평균</b>',
        line=dict(color='firebrick', width=4),
        hovertemplate='<b>%{x|%Y-%m} 통합 평균: %{y:,.0f}만 원</b><extra></extra>'
    ))

    # 6. 전체 디자인 및 X축 날짜 설정
    fig.update_layout(
        title=dict(
            text='<b>강남 3구(강남·송파·서초) 및 통합 월별 가격 추이 비교</b>',
            x=0.5, font=dict(size=22)
        ),
        width=1200, height=700,
        xaxis=dict(title=dict(text='계약월', standoff=30)),
        yaxis_title='평균 거래가 (만 원)',
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5
        ),
    )

    # X축 설정 업데이트
    start_date = plot_gu.index.min()
    end_date = plot_gu.index.max()
    fig.update_xaxes(
        range=[start_date, end_date],
        tickvals=plot_gu.index,
        ticktext=[f"{m.month}월" for m in plot_gu.index],
        tickformat="%m월", 
        dtick="M1", 
        tickangle=0,
    )

    # 연도 구분선 추가
    years = [2024, 2025, 2026]
    for year in years:
        year_start = f"{year}-01-01"
        if plot_gu.index.min() <= pd.to_datetime(year_start) <= plot_gu.index.max():
            fig.add_vline(
                x=year_start,
                line_width=1.5,
                line_dash="solid",
                line_color="rgba(200, 200, 200, 0.5)"
            )
            fig.add_annotation(
                x=year_start, y=1, yref="paper",
                text=f"<b>{year}년</b>",
                showarrow=False, font=dict(color="gray", size=12),
                xanchor="center", yanchor="bottom"
            )

    # Y축 숫자 콤마 설정
    fig.update_yaxes(tickformat=',d')

    return fig # [추가] 완성된 그래프 객체를 결과값으로 돌려줍니다.

# --- 실제 사용 예시 ---
# 1. 함수를 실행하고 그 결과를 'my_chart'라는 변수에 저장합니다.
my_chart = draw_trend_graph(trend_gu_monthly) 

# 2. 저장된 변수를 호출하여 그래프를 보여줍니다.
my_chart.show()

## 왜 송파구만 저렇게 거래가가 낮지?

### 송파구 vs (강남구, 서초구) 비교해보자! 

In [ ]:
# # 1. 대상 구(강남, 송파, 서초) 필터링
# targets = ['강남구', '송파구', '서초구']
# gu_comparison = df[df['자치구명'].isin(targets)].copy()

# 2. 구별 거래 건수(Count), 평균 금액(Mean), 합계 금액(Sum) 비교
# MECE하게 분석하기 위해 합계까지 포함하여 비교합니다.
comparison_stats = df.groupby('자치구명').agg(
    거래건수=('물건금액', 'count'),      # 얼마나 자주 거래되었나 (Volume)
    평균가=('물건금액', 'mean'),        # 전형적인 가격 수준 (Average)
    합계금액=('물건금액', 'sum')         # 해당 지역 전체 자금 규모 (Total Amount)
).round(0)

# 3. 거래 건수 순으로 정렬하여 보기
print("강남 3구 거래 규모 및 가격 비교")
print(comparison_stats.sort_values(by='거래건수', ascending=False))

### 송파 vs (강남, 서초) 비교 데이터 시각화 

In [ ]:
import plotly.express as px # 시각화 라이브러리를 가져옵니다.

# 1. 인덱스 변환
plot_stats = comparison_stats.reset_index() # 인덱스를 컬럼으로 바꿉니다.

# 2. 거래건수 시각화 (시장 활성도)
fig_count = px.bar(
    plot_stats, 
    x='자치구명', 
    y='거래건수', 
    text_auto=True, # 숫자를 막대 위에 표시합니다.
    title='<b>강남 3구 지역별 거래건수 (Volume)</b>', # 제목을 설정합니다.
    color='자치구명', # 구별로 색상을 나눕니다.
    template='plotly_white' # 흰색 배경을 사용합니다.
)

# [수정] 거래건수 그래프 레이아웃 최적화
fig_count.update_layout(
    width=600, # 너비를 600으로 줄입니다.
    height=450, # 높이를 450으로 줄입니다.
    title=dict(font=dict(size=20), x=0.5), # 제목 크기를 20으로 줄이고 중앙에 둡니다.
    xaxis=dict(title="", tickfont=dict(size=14)), # X축 제목을 없애고 글자 크기를 조정합니다.
    yaxis=dict(title="거래건수 (건)", tickfont=dict(size=12), tickformat=",d"), # Y축 숫자 포맷을 설정합니다.
    showlegend=False, # 범례를 숨깁니다.
    margin=dict(l=60, r=20, t=80, b=40), # 여백을 줄여 막대 영역을 넓힙니다.
    bargap=0.4 # 막대 사이의 간격을 적절히 벌립니다.
)
fig_count.show() # 그래프를 출력합니다.

# 3. 평균가 시각화 (가격 수준)
fig_mean = px.bar(
    plot_stats, 
    x='자치구명', 
    y='평균가', 
    text_auto=',.0f', # 천 단위 콤마를 표시합니다.
    title='<b>강남 3구 지역별 평균 거래가 (Average Price)</b>', # 제목을 설정합니다.
    color='자치구명', # 색상을 지정합니다.
    template='plotly_white' # 배경 스타일입니다.
)

# [수정] 평균가 그래프 레이아웃 최적화
fig_mean.update_layout(
    width=600, # 너비를 600으로 줄입니다.
    height=450, # 높이를 450으로 줄입니다.
    title=dict(font=dict(size=20), x=0.5), # 제목 크기를 20으로 줄이고 중앙 배치합니다.
    xaxis=dict(title="", tickfont=dict(size=14)), # X축 설정을 조정합니다.
    yaxis=dict(title="평균가 (만 원)", tickfont=dict(size=12), tickformat=",d"), # Y축 숫자 포맷을 설정합니다.
    showlegend=False, # 범례를 숨깁니다.
    margin=dict(l=60, r=20, t=80, b=40), # 여백을 최적화합니다.
    bargap=0.4 # 막대 두께를 조절합니다.
)
fig_mean.show() # 그래프를 출력합니다.

#### 아! 거래총액이 2위인걸로 봐서 평균가격이 낮은 이유는 거래 건수는 많은데 거래가가 낮아서 그렇구나! 
####그럼 이 거래가가 왜 낮지? 혹시 평형수가 작은 아파트들이 강남, 서초에 비해 많은가? 

In [ ]:
# 1. 현재 df가 가진 모든 컬럼명을 한 줄로 출력합니다.
print("실제 컬럼명 리스트:", df.columns.tolist()) # 이 출력 결과에서 면적 관련 이름을 찾아야 합니다.

#2. 건물면적을 '평수'로 계산한 컬럼 생성
df['평수'] = df['건물면적'] / 3.3058 
print(df['평수'].head().round(2))

# 3. 평형대 구간(Bin) 나누기
# 0~10평, 10~20평, 20~30평, 30~40평, 40평 이상으로 구분합니다.
bins = [0, 10, 20, 30, 40, 1000] # 구간의 경계값을 설정합니다. # (1000은 40평 이상의 모든 값 포함)
labels = ['10평 이하', '10평대', '20평대', '30평대', '40평 이상'] # 각 구간에 붙일 이름표입니다.

# 4. 평형구분 컬럼 생성
df['평형구분'] = pd.cut(df['평수'], bins=bins, labels=labels, right=False) # 평수 데이터를 기준에 따라 구간별로 분류합니다. # (cut 함수 사용)

# 5. 강남 3구 필터링 및 구별/평형별 거래건수 집계
targets = ['강남구', '송파구', '서초구'] # 비교할 대상 구를 리스트로 만듭니다.
comparison_df = df[df['자치구명'].isin(targets)].copy() # 대상 구의 데이터만 복사해옵니다.

# 6. 거래건수 확인 (물건금액 컬럼의 개수를 카운트)
# 자치구명과 평형구분을 기준으로 묶어 건수를 계산합니다.
result = comparison_df.groupby(['자치구명', '평형구분']).agg(
    거래건수=('물건금액', 'count') # 각 그룹별 물건금액의 개수를 세어 거래건수를 구합니다. # (Aggregation)
).reset_index() # 결과표를 보기 좋게 평면 데이터로 펼칩니다.

# 7. 결과 출력
print("[강남 3구 평형대별 거래건수 비교]")
print(result.pivot(index='평형구분', columns='자치구명', values='거래건수')) # 가로축을 구, 세로축을 평형으로 하는 표로 변환합니다.

In [ ]:
import plotly.express as px  # 시각화 라이브러리를 가져옵니다.

# 1. 10평 이하 데이터 제거 (Filtering)
plot_df_filtered = plot_df[plot_df['평형구분'] != '10평 이하'].copy() # '10평 이하' 제외 데이터를 추출합니다.

# 2. 각 자치구 내에서의 비율(%) 계산
total_counts = plot_df_filtered.groupby('자치구명')['거래건수'].transform('sum') # 구별 총합을 구합니다.
plot_df_filtered['비율'] = (plot_df_filtered['거래건수'] / total_counts * 100).round(1) # 백분율을 구합니다.

# 3. 그래프 라벨 생성 (좁은 공간에 맞춰 <br> 태그로 줄바꿈 적용)
plot_df_filtered['라벨'] = plot_df_filtered.apply(
    lambda row: f"{row['비율']}%<br>({int(row['거래건수']):,}건)", axis=1 # 작은 공간에서도 잘 보이도록 줄바꿈합니다.
)

# 4. 누적 막대 그래프 생성 (색상 팔레트 적용)
fig = px.bar(
    plot_df_filtered, # 필터링된 데이터를 사용합니다.
    x='자치구명', # 가로축입니다.
    y='거래건수', # 세로축입니다.
    color='평형구분', # 평형대별 색상 구분입니다.
    text='라벨', # 생성한 라벨을 표시합니다.
    title='<b>강남 3구 평형대별 거래 비중 및 규모</b>', # 제목을 조금 더 간결하게 수정합니다.
    color_discrete_sequence=px.colors.sequential.Blues_r, # 진한 파랑에서 연한 파랑 순서입니다.
    template='plotly_white' # 배경을 흰색으로 설정합니다.
)

# 5. 축소된 사이즈(600x450)에 최적화된 레이아웃 업데이트
fig.update_layout(
    # 제목 사이즈를 전체 비율에 맞춰 조정 (28 -> 20)
    title=dict(font=dict(size=20), x=0.5, y=0.95), 
    
    # 요청하신 슬림한 사이즈 적용
    width=600, # 그래프 너비입니다.
    height=450, # 그래프 높이입니다.
    
    # 좁은 화면에서 막대가 너무 얇아지지 않도록 간격 조정 (0.4 -> 0.3)
    bargap=0.3, 
    
    # X축 텍스트 사이즈 조정 (16 -> 14)
    xaxis=dict(title="", tickfont=dict(size=14)), 
    
    # Y축 숫자 포맷 및 사이즈 조정 (14 -> 12)
    yaxis=dict(
        title="거래건수 (건)", 
        tickformat=",d", 
        tickfont=dict(size=12),
        range=[0, plot_df_filtered.groupby('자치구명')['거래건수'].sum().max() * 1.1] # 상단 여백 확보
    ), 
    
    # 범례(Legend) 사이즈 축소 및 배치 최적화
    legend_title=dict(text="평형대", font=dict(size=12)),
    legend=dict(font=dict(size=10), x=1.02, y=1), # 범례를 살짝 오른쪽으로 밀어 본문 공간 확보
    
    # 여백 최적화 (여백을 줄여 막대 영역을 넓힘)
    margin=dict(l=60, r=20, t=80, b=40)
)

# 막대 내부 텍스트 스타일 및 가독성 최적화
fig.update_traces(
    textposition='inside', # 텍스트를 막대 안에 넣습니다.
    textfont_size=10, # 작은 막대에서도 글자가 깨지지 않도록 사이즈 축소 (12 -> 10)
    textfont_color='white', # 파란색 배경에서 잘 보이도록 흰색 유지
    insidetextanchor='middle' # 텍스트를 막대 칸 중앙에 배치합니다.
)

fig.show() # 최종 결과물을 출력합니다.

In [ ]:
my_chart.show()  

## ==> 박스 플롯으로 강남, 서초에 있을 이상치? 알아보기! 박스플롯으로 했을 때 더 적합할 것 같음 !
## ==> 해석 방법? 을 AI한테 물어보기! 


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# 1. 분석 대상 설정: 2025년 4월 & 강남 3구 데이터 필터링
target_month = '2025-04'
targets = ['강남구', '서초구', '송파구']

# '계약일'에서 연-월 정보를 추출하여 필터링
analysis_df = df[(df['계약일'].dt.to_period('M') == target_month) & 
                 (df['자치구명'].isin(targets))].copy()

# 2. 자치구별 통계 요약 함수 (MECE하게 정보 추출)
def get_box_stats(data, gu_name):
    gu_df = data[data['자치구명'] == gu_name]
    q1 = gu_df['물건금액'].quantile(0.25)
    q3 = gu_df['물건금액'].quantile(0.75)
    iqr = q3 - q1
    upper_fence = q3 + (1.5 * iqr)
    outliers = gu_df[gu_df['물건금액'] > upper_fence]
    
    return {
        '구': gu_name,
        '중앙값': gu_df['물건금액'].median(),
        '평균': gu_df['물건금액'].mean(),
        '이상치경계': upper_fence,
        '이상치건수': len(outliers)
    }

# 각 구별 통계 출력
stats_list = [get_box_stats(analysis_df, gu) for gu in targets]
stats_summary = pd.DataFrame(stats_list)
print(f"--- {target_month} 강남 3구 4분위 통계 비교 ---")
print(stats_summary)

# 3. 박스플롯 시각화 (병렬 비교)
fig = px.box(analysis_df, 
             x="자치구명", 
             y="물건금액", 
             color="자치구명",
             points="all",  # 모든 거래 데이터를 점으로 표시
             title=f"<b>{target_month} 강남 3구 매매가 분포 비교 (Box Plot)</b>",
             category_orders={"자치구명": ["강남구", "서초구", "송파구"]},
             color_discrete_map={'강남구': 'royalblue', '서초구': 'darkorange', '송파구': 'forestgreen'},
             labels={'물건금액':'거래금액(만 원)', '자치구명':''},
             template='plotly_white')

# 디자인 최적화
fig.update_layout(
    showlegend=False,
    width=900, height=600,
    yaxis_tickformat=',d',
    font=dict(size=14)
)

fig.show()

In [ ]:
#분석 구현 코드 (강남 vs 서초 4월 정밀 비교)
#두 구의 데이터 분포를 직접 비교하여 "서초구는 왜 하락했는가"를 보여주는 시각화 코드입니다.

import plotly.express as px

# 1. 2025년 4월 강남구와 서초구 데이터만 추출
target_month = '2025-04'
comparison_df = df[(df['계약일'].dt.to_period('M') == target_month) & 
                    (df['자치구명'].isin(['강남구', '서초구']))].copy()

# 2. 박스플롯 시각화 (서초구의 하락 원인 파악용)
fig = px.box(comparison_df, x="자치구명", y="물건금액", color="자치구명",
             points="all", # 개별 거래 점을 모두 표시하여 거래 분포 확인
             title=f"<b>{target_month} 강남구 vs 서초구 가격 분포 비교</b>",
             color_discrete_map={'강남구': 'royalblue', '서초구': 'darkorange'},
             template='plotly_white')

# 3. 평균값 비교선 추가
for gu in ['강남구', '서초구']:
    avg_val = comparison_df[comparison_df['자치구명'] == gu]['물건금액'].mean()
    fig.add_hline(y=avg_val, line_dash="dot", line_color='gray', 
                  annotation_text=f"{gu} 평균")

fig.show()

# 4. (추가) 거래량 수치 확인
print(comparison_df.groupby('자치구명')['물건금액'].agg(['count', 'mean', 'median']))

In [ ]:
import pandas as pd
import plotly.express as px

# 1. 2025년 4월 강남 3구 데이터 추출
target_month = '2025-04'
targets = ['강남구', '서초구', '송파구']
df_apr = df[(df['계약일'].dt.to_period('M') == target_month) & 
            (df['자치구명'].isin(targets))].copy()

# 2. 동별 집계 (평균가, 거래건수, 최고가)
dong_summary = df_apr.groupby(['자치구명', '법정동명'])['물건금액'].agg(
    평균거래가='mean',
    거래건수='count',
    최고가='max'
).reset_index()

# 3. 버블 차트 시각화 (Price vs Volume Analysis)
# 원의 크기가 클수록 거래 건수가 많음을 의미합니다.
fig = px.scatter(dong_summary, 
                 x="거래건수", 
                 y="평균거래가", 
                 size="거래건수", 
                 color="자치구명",
                 hover_name="법정동명",
                 text="법정동명",
                 title=f"<b>{target_month} 강남 3구 동별 거래가-거래량 분석</b>",
                 labels={'평균거래가':'평균 실거래가 (만 원)', '거래건수':'거래 건수 (건)'},
                 color_discrete_map={'강남구': 'royalblue', '서초구': 'darkorange', '송파구': 'forestgreen'},
                 template='plotly_white',
                 size_max=40)

# 텍스트 위치 및 디자인 조정
fig.update_traces(textposition='top center')
fig.update_layout(
    width=1000, height=700,
    yaxis_tickformat=',d',
    xaxis_title="거래 건수 (Volume)",
    yaxis_title="평균 실거래가 (Price)",
    font=dict(size=12)
)

fig.show()

# 4. 상세 데이터 출력 (Top 10 거래가 순)
print(f"--- {target_month} 강남 3구 동별 상세 현황 ---")
print(dong_summary.sort_values(by='평균거래가', ascending=False).head(15))